In [1]:
# import dependencies and keys
import pandas as pd
import gmaps
import requests
#key
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# read cities.csv into df
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Ushuaia,AR,1/27/2022 16:31,-54.8000,-68.3000,49.66,71,75,19.57
1,1,Zenzeli,RU,1/27/2022 16:31,45.9241,47.0474,29.82,93,100,5.99
2,2,Marrakesh,MA,1/27/2022 16:31,31.6315,-8.0083,66.27,37,75,5.75
3,3,Pierre,US,1/27/2022 16:31,44.3683,-100.3510,34.86,76,100,7.00
4,4,Rikitea,PF,1/27/2022 16:31,-23.1203,-134.9692,78.08,77,27,19.24


In [3]:
# check the data types within the dataframe
city_data_df.dtypes


City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# deal with negative numbers for max temps
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))


In [5]:
# Create the heatmap for max temp
locations = city_data_df[['Lat', 'Lng']]
max_temp = city_data_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create a heat map for humidity
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Create a heat map for Cloudiness
locations = city_data_df[['Lat', 'Lng']]
cloudiness = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Create a heat map for Cloudiness
locations = city_data_df[['Lat', 'Lng']]
wind_speed = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip?'))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip?85


In [13]:
# filter the dataset to find the cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &  (city_data_df['Max Temp'] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Marrakesh,MA,1/27/2022 16:31,31.6315,-8.0083,66.27,37,75,5.75
4,4,Rikitea,PF,1/27/2022 16:31,-23.1203,-134.9692,78.08,77,27,19.24
8,8,Praia,CV,1/27/2022 16:31,14.9215,-23.5087,75.74,60,75,25.32
12,12,Road Town,VG,1/27/2022 16:29,18.4167,-64.6167,84.29,61,40,11.50
14,14,Vaini,TO,1/27/2022 16:31,-21.2000,-175.2000,81.84,80,97,15.61
15,15,Vallenar,CL,1/27/2022 16:31,-28.5708,-70.7581,83.12,36,0,10.51
16,16,Ambon,ID,1/27/2022 16:31,-3.6954,128.1814,77.14,88,20,5.75
17,17,Grand Gaube,MU,1/27/2022 16:31,-20.0064,57.6608,77.40,71,22,3.00
20,20,Port Lincoln,AU,1/27/2022 16:31,-34.7333,135.8667,65.14,89,0,10.13
21,21,Georgetown,MY,1/27/2022 16:26,5.4112,100.3354,82.33,82,20,3.44


In [14]:
# determine if there are null values in the dataframe
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [15]:
# create a hotel dataframe to store hotel names along with the city, max temp, coordinates, and country
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Marrakesh,MA,66.27,31.6315,-8.0083,
4,Rikitea,PF,78.08,-23.1203,-134.9692,
8,Praia,CV,75.74,14.9215,-23.5087,
12,Road Town,VG,84.29,18.4167,-64.6167,
14,Vaini,TO,81.84,-21.2000,-175.2000,
15,Vallenar,CL,83.12,-28.5708,-70.7581,
16,Ambon,ID,77.14,-3.6954,128.1814,
17,Grand Gaube,MU,77.40,-20.0064,57.6608,
20,Port Lincoln,AU,65.14,-34.7333,135.8667,
21,Georgetown,MY,82.33,5.4112,100.3354,


In [16]:
# set parameters to search for a hotel
parameters = {'radius': 5000, 'type': 'lodging', 'key': g_key}
#iterate through the hotel_df
for index, row in hotel_df.iterrows():
    #get lat and lng
    lat = row['Lat']
    lng = row['Lng']
    #add lat and lng to the location key of parameters dictionary
    parameters['location'] = f'{lat}, {lng}'
    # use the search term lodging and our lat and long
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #make the request and format as json
    hotels = requests.get(base_url, params=parameters).json()
    #grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError): 
        print('Hotel not found......skipping')
    

Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping
Hotel not found......skipping


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Marrakesh,MA,66.27,31.6315,-8.0083,Novotel Marrakech Hivernage
4,Rikitea,PF,78.08,-23.1203,-134.9692,People ThankYou
8,Praia,CV,75.74,14.9215,-23.5087,LT Aparthotel
12,Road Town,VG,84.29,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
14,Vaini,TO,81.84,-21.2000,-175.2000,Keleti Beach Resort
15,Vallenar,CL,83.12,-28.5708,-70.7581,Humacao Bed & Breakfast
16,Ambon,ID,77.14,-3.6954,128.1814,Swiss-Belhotel Ambon
17,Grand Gaube,MU,77.40,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
20,Port Lincoln,AU,65.14,-34.7333,135.8667,Port Lincoln Hotel
21,Georgetown,MY,82.33,5.4112,100.3354,Cititel Penang


In [20]:
#create hotel df heatmap
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer=gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
#create info box template
info_box_template = ''' 
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} F°</dd>
</dl>
'''
#Store the dataframe row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]